查看cuda版本  cuda>=10.0

In [ ]:
!cat /usr/local/cuda/version.txt

CUDA Version 10.1.243


cuda<10.0,则执行以下代码

In [ ]:

!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget http://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
!sudo apt-key add /var/cuda-repo-10-2-local-10.2.89-440.33.01/7fa2af80.pub
!sudo apt-get update
!sudo apt-get -y install cuda=10.2.89-1

--2020-07-04 08:23:16--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu1804.pin’

cuda-ubuntu1804.pin 100%[===================>]     190  --.-KB/s    in 0s      

2020-07-04 08:23:16 (4.23 MB/s) - ‘cuda-ubuntu1804.pin’ saved [190/190]

--2020-07-04 08:23:24--  http://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:80... connected.
HTTP request sent, awaiting response...

安装mxnet和gluonnlp

In [ ]:
!pip install --upgrade mxnet-cu100==1.6.0 -f https://dist.mxnet.io/python

Looking in links: https://dist.mxnet.io/python
     |████████████████████████████████| 695.0MB 27kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [ ]:
!pip install --upgrade gluonnlp

     |████████████████████████████████| 256kB 9.2MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp36-cp36m-linux_x86_64.whl size=470049 sha256=65eae1c69da4fc3f46dd62f56b12e9674766ab4e7232d3c996de9c4fdd21dba1
  Stored in directory: /root/.cache/pip/wheels/af/60/16/1f8a40e68b85bd9bd7960e91830bca5e40cd113f3220b7e231
Successfully built gluonnlp


查看硬件gpu信息

In [ ]:
!nvidia-smi

Sun Jul 12 10:36:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

加载google云端硬盘

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


根据文件路径进行修改

In [ ]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks")

In [ ]:
cd NLP/

/content/gdrive/My Drive/Colab Notebooks/NLP


导入相关包

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import io
import random
import numpy as np
import mxnet as mx
import gluonnlp as nlp
from gluonnlp.calibration import BertLayerCollector
# this notebook assumes that all required scripts are already
# downloaded from the corresponding tutorial webpage on http://gluon-nlp.mxnet.io
from bert import data


In [ ]:
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)
# change `ctx` to `mx.cpu()` if no GPU is available.
ctx = mx.gpu(0)

In [ ]:
#加载模型
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12',
                                             dataset_name='book_corpus_wiki_en_uncased',
                                             pretrained=True, ctx=ctx, use_pooler=True,
                                             use_decoder=False, use_classifier=False)

Vocab file is not found. Downloading.


In [ ]:
bert_classifier = nlp.model.BERTClassifier(bert_base, num_classes=2, dropout=0.1)
# only need to initialize the classifier layer. 添加分类曾->初始化->hybrid混合化,提高训练效率(MXNet最重要的部分)
bert_classifier.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
bert_classifier.hybridize(static_alloc=True)

# softmax cross entropy loss for classification  损失函数与评估函数
loss_function = mx.gluon.loss.SoftmaxCELoss()
loss_function.hybridize(static_alloc=True)

metric = mx.metric.Accuracy()

In [ ]:
#使用nlp.data.TSVDataset来读取tsv文件 
# Skip the first line, which is the schema跳过第一行
num_discard_samples = 1
field_separator = nlp.data.Splitter('\t')
sample_splitter = nlp.data.Splitter('\n')
# Fields to select from the file
field_indices = [2,1]#取句子  标签
data_train_raw = nlp.data.TSVDataset(filename='labeledTrainData.tsv',
                                 sample_splitter=sample_splitter,
                                 field_separator=field_separator,
                                 num_discard_samples=num_discard_samples,
                                 field_indices=field_indices,
                                )
sample_id = 100
print(data_train_raw[sample_id][0])
# label
print(data_train_raw[sample_id][1])

"There is a uk edition to this show which is rather less extravagant than the US version. The person concerned will get a new kitchen or perhaps bedroom and bathroom and is wonderfully grateful for what they have got. The US version of this show is everything that reality TV shouldn't be. Instead of making a few improvements to a house which the occupants could not afford or do themselves the entire house gets rebuilt. I do not know if this show is trying to show what a lousy welfare system exists in the US or if you beg hard enough you will receive. The rather vulgar product placement that takes place, particularly by Sears, is also uncalled for. Rsther than turning one family in a deprived area into potential millionaires, it would be far better to help the community as a whole where instead of spending the hundreds of thousands of dollars on one home, build something for the whole community ..... perhaps a place where diy and power tools can be borrowed and returned along with build

In [ ]:
# Use the vocabulary from pre-trained model for tokenization 使用BERTDatasetTransform来实现数据格式变换
bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True) #使用加载的预训练模型的词典

# The maximum length of an input sequence  最大句子长度
max_len = 128

# The labels for the two classes
all_labels = ["0", "1"]

# 对于单个句子分类问题,pair=False  我们只需要改这个
# 对于回归任务,class_labels=False
# 对于推理,has_label=False
pair = False
transform = data.transform.BERTDatasetTransform(bert_tokenizer, max_len,
                                                class_labels=all_labels,
                                                has_label=True,
                                                pad=True,
                                                pair=pair)
data_train = data_train_raw.transform(transform)
# 输出变换后的句子
print('vocabulary used for tokenization = \n%s'%vocabulary)
print('%s token id = %s'%(vocabulary.padding_token, vocabulary[vocabulary.padding_token]))
print('%s token id = %s'%(vocabulary.cls_token, vocabulary[vocabulary.cls_token]))
print('%s token id = %s'%(vocabulary.sep_token, vocabulary[vocabulary.sep_token]))
print('token ids = \n%s'%data_train[sample_id][0])
print('segment ids = \n%s'%data_train[sample_id][1])
print('valid length = \n%s'%data_train[sample_id][2])
print('label = \n%s'%data_train[sample_id][3])

vocabulary used for tokenization = 
Vocab(size=30522, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
[PAD] token id = 1
[CLS] token id = 2
[SEP] token id = 3
token ids = 
[    2  1000  2045  2003  1037  2866  3179  2000  2023  2265  2029  2003
  2738  2625 27856  2084  1996  2149  2544  1012  1996  2711  4986  2097
  2131  1037  2047  3829  2030  3383  5010  1998  5723  1998  2003  6919
  2135  8794  2005  2054  2027  2031  2288  1012  1996  2149  2544  1997
  2023  2265  2003  2673  2008  4507  2694  5807  1005  1056  2022  1012
  2612  1997  2437  1037  2261  8377  2000  1037  2160  2029  1996 18837
  2071  2025  8984  2030  2079  3209  1996  2972  2160  4152  7183  1012
  1045  2079  2025  2113  2065  2023  2265  2003  2667  2000  2265  2054
  1037 10223  6508  7574  2291  6526  1999  1996  2149  2030  2065  2017
 11693  2524  2438  2017  2097  4374  1012  1996  2738 29364  4031 11073
  2008  3138  2173  1010  3391  2011 18493     3]
segment ids = 
[0 0 0 0 0 0 0 0 0

In [ ]:

# The hyperparameters
batch_size = 16
lr = 5e-6

# The FixedBucketSampler and the DataLoader for making the mini-batches
train_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[2]) for item in data_train],
                                            batch_size=batch_size,
                                            shuffle=True)
bert_dataloader = mx.gluon.data.DataLoader(data_train, batch_sampler=train_sampler)

trainer = mx.gluon.Trainer(bert_classifier.collect_params(), 'adam',
                           {'learning_rate': lr, 'epsilon': 1e-9})

# Collect all differentiable parameters
# `grad_req == 'null'` indicates no gradients are calculated (e.g. constant parameters)
# The gradients for these params are clipped later
params = [p for p in bert_classifier.collect_params().values() if p.grad_req != 'null']
grad_clip = 1

# Training the model with only three epochs
log_interval = 100
num_epochs = 5
for epoch_id in range(num_epochs):
    metric.reset()
    step_loss = 0
    for batch_id, (token_ids, segment_ids, valid_length, label) in enumerate(bert_dataloader):
        with mx.autograd.record():

            # Load the data to the GPU
            token_ids = token_ids.as_in_context(ctx)
            valid_length = valid_length.as_in_context(ctx)
            segment_ids = segment_ids.as_in_context(ctx)
            label = label.as_in_context(ctx)

            # 前馈运算 -> forward
            out = bert_classifier(token_ids, segment_ids, valid_length.astype('float32'))
            # 做均值运算
            ls = loss_function(out, label).mean()

        # And backwards computation
        ls.backward()

        # 梯度截断
        trainer.allreduce_grads()# 梯度累加
        nlp.utils.clip_grad_global_norm(params, 1)# 门限为1
        trainer.update(1) # 因为ls已经mean()了,所以这里的batch_size=1

        step_loss += ls.asscalar()
        metric.update([label], [out])

        # Printing vital information
        if (batch_id + 1) % (log_interval) == 0:
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.7f}, acc={:.3f}'
                         .format(epoch_id, batch_id + 1, len(bert_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, metric.get()[1]))
            step_loss = 0
    #导出模型
    filename="./base_"+str(epoch_id)+".params"
    nlp.utils.save_parameters(bert_classifier,filename)

[Epoch 0 Batch 100/1567] loss=0.2583, lr=0.0000050, acc=0.900
[Epoch 0 Batch 200/1567] loss=0.2872, lr=0.0000050, acc=0.894
[Epoch 0 Batch 300/1567] loss=0.2574, lr=0.0000050, acc=0.898
[Epoch 0 Batch 400/1567] loss=0.2693, lr=0.0000050, acc=0.895
[Epoch 0 Batch 500/1567] loss=0.2497, lr=0.0000050, acc=0.896
[Epoch 0 Batch 600/1567] loss=0.2555, lr=0.0000050, acc=0.896
[Epoch 0 Batch 700/1567] loss=0.2272, lr=0.0000050, acc=0.898
[Epoch 0 Batch 800/1567] loss=0.2432, lr=0.0000050, acc=0.898
[Epoch 0 Batch 900/1567] loss=0.2781, lr=0.0000050, acc=0.897
[Epoch 0 Batch 1000/1567] loss=0.2458, lr=0.0000050, acc=0.898
[Epoch 0 Batch 1100/1567] loss=0.2329, lr=0.0000050, acc=0.899
[Epoch 0 Batch 1200/1567] loss=0.2358, lr=0.0000050, acc=0.900
[Epoch 0 Batch 1300/1567] loss=0.2752, lr=0.0000050, acc=0.899
[Epoch 0 Batch 1400/1567] loss=0.2366, lr=0.0000050, acc=0.899
[Epoch 0 Batch 1500/1567] loss=0.2537, lr=0.0000050, acc=0.899
[Epoch 1 Batch 100/1567] loss=0.2114, lr=0.0000050, acc=0.924
[E

In [ ]:
#加载验证集
# Skip the first line, which is the schema
num_discard_samples = 1
# Split fields by tabs
field_separator = nlp.data.Splitter('\t')
sample_splitter = nlp.data.Splitter('\n')
# Fields to select from the file
field_indices=[2,1]
data_test_raw = nlp.data.TSVDataset(filename='testDataset.tsv',
                  sample_splitter=sample_splitter,
                  field_separator=field_separator,
                  num_discard_samples=num_discard_samples,
                  field_indices=field_indices)
data_test = data_test_raw.transform(transform)

In [ ]:
#在验证集上的精度
num_epochs = 5
batch_size = 16
test_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[2]) for item in data_test],
                      batch_size=batch_size,
                      shuffle=True)
test_dataloader = mx.gluon.data.DataLoader(data_test, batch_sampler=test_sampler)

validation = nlp.model.BERTClassifier(bert_base, num_classes=2, dropout=0.1)
validation.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
validation.hybridize(static_alloc=True)

for epoch_id in range(num_epochs):
  filename="./base_"+str(epoch_id)+".params"
  nlp.utils.load_parameters(validation,filename,ctx=ctx)
  metric.reset()
  for batch_id, (token_ids, segment_ids, valid_length, label) in enumerate(test_dataloader):
    with mx.autograd.record():
      # Load the data to the GPU
      token_ids = token_ids.as_in_context(ctx)
      valid_length = valid_length.as_in_context(ctx)
      segment_ids = segment_ids.as_in_context(ctx)
      label = label.as_in_context(ctx)

      # Forward computation
      out = validation(token_ids, segment_ids, valid_length.astype('float32'))
    metric.update([label], [out])
  print('Epoch '+str(epoch_id)+' validation acc:',metric.get()[1])


Epoch 0 validation acc: 0.8752
Epoch 1 validation acc: 0.87596
Epoch 2 validation acc: 0.877
Epoch 3 validation acc: 0.85704
Epoch 4 validation acc: 0.86244


选择epoch 2的模型，acc最高